人工合成数据集

In [ ]:
# Make an example with 2D latent states and 4 discrete states
def simulate_nascar():
    assert K_true == 4

    def random_rotation(n, theta):
        rot = np.array([[np.cos(theta), -np.sin(theta)],
                        [np.sin(theta), np.cos(theta)]])
        out = np.zeros((n,n))
        out[:2,:2] = rot
        q = np.linalg.qr(np.random.randn(n,n))[0]
        # q = np.eye(n)
        return q.dot(out).dot(q.T)

    As = [random_rotation(D_latent, np.pi/24.),
          random_rotation(D_latent, np.pi/48.)]

    # Set the center points for each system
    centers = [np.array([+2.0, 0.]),
               np.array([-2.0, 0.])]
    bs = [-(A - np.eye(D_latent)).dot(center) for A, center in zip(As, centers)]

    # Add a "right" state
    As.append(np.eye(D_latent))
    bs.append(np.array([+0.1, 0.]))

    # Add a "right" state
    As.append(np.eye(D_latent))
    bs.append(np.array([-0.35, 0.]))

    # Construct multinomial regression to divvy up the space #
    tree = decision_list(K_true)
    w1, b1 = np.array([+1.0, 0.0]), np.array([-2.0])   # x + b > 0 -> x > -b
    w2, b2 = np.array([-1.0, 0.0]), np.array([-2.0])   # -x + b > 0 -> x < b
    w3, b3 = np.array([0.0, +1.0]), np.array([0.0])    # y > 0

    reg_W = np.row_stack((w1, w2, w3))
    reg_b = np.row_stack((b1, b2, b3))

    # Scale the weights to make the transition boundary sharper
    reg_scale = 100.
    reg_b *= reg_scale
    reg_W *= reg_scale

    # Account for stick breaking asymmetry
    mu_b, _ = compute_psi_cmoments(np.ones(K_true))
    reg_b += mu_b[:,None]

    # Make a recurrent SLDS with these params #
    dynamics_distns = [
        Regression(
            A=np.column_stack((A,b)),
            sigma=1e-4 * np.eye(D_latent),
            nu_0=D_latent + 2,
            S_0=1e-4 * np.eye(D_latent),
            M_0=np.zeros((D_latent, D_latent + 1)),
            K_0=np.eye(D_latent + 1),
        )
        for A,b in zip(As, bs)]

    init_dynamics_distns = [
        Gaussian(
            mu=np.array([0.0, 1.0]),
            sigma=1e-3 * np.eye(D_latent))
        for _ in range(K_true)]

    C = np.hstack((npr.randn(args.D_obs, D_latent), np.zeros((args.D_obs, 1))))
    emission_distns = \
        DiagonalRegression(args.D_obs, D_latent+1,
                           A=C, sigmasq=1e-5 *np.ones(args.D_obs),
                           alpha_0=2.0, beta_0=2.0)

    model = PGRecurrentSLDS(
        trans_params=dict(A=np.hstack((np.zeros((K_true-1, K_true)), reg_W)), b=reg_b,
                          sigmasq_A=100., sigmasq_b=100., tree=tree),
        init_state_distn='uniform',
        init_dynamics_distns=init_dynamics_distns,
        dynamics_distns=dynamics_distns,
        emission_distns=emission_distns)

    # Sample from the model
    inputs = np.ones((args.T, 1))
    y, x, z = model.generate(T=args.T, inputs=inputs)

    # Maks off some data
    mask = np.ones((args.T, args.D_obs), dtype=bool)
    mask[args.mask_start:args.mask_stop] = False
    return model, inputs, z, x, y, mask